# Grid4x4 - PettingZoo + RLlib

In [1]:
import os
import ray

In [2]:
import random
import numpy as np
import torch

SEED = 23423  # default SUMO seed no.
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

## Environment Setup

In [3]:
import supersuit as ss

from helper_functions import make_parallel_env
from observation import Grid4x4ObservationFunction
from reward_functions import combined_reward

def env_creator(args):
    env_params = {
        "net_file": os.path.join("nets","grid4x4","grid4x4.net.xml"),
        "route_file": os.path.join("nets","grid4x4","grid4x4_1.rou.xml"),
        "num_seconds": 3600,
        "reward_fn": combined_reward,
        "sumo_seed": SEED,
        # "observation_class": Grid4x4ObservationFunction
    }
    env = make_parallel_env(**env_params)
    env = ss.frame_stack_v1(env, 3)
    return env

## Training the RL Agent

In [4]:
from ray.rllib.env.wrappers.pettingzoo_env import ParallelPettingZooEnv  # RLlib-PZ interface
from ray.tune.registry import register_env

ray.init()

env_name = "grid4x4"
register_env(env_name, lambda config: ParallelPettingZooEnv(env_creator(config)))

2023-05-29 12:14:59,049	INFO worker.py:1625 -- Started a local Ray instance.


In [5]:
from ray.rllib.algorithms.ppo import PPOConfig

# From https://github.com/ray-project/ray/blob/master/rllib/tuned_examples/ppo/atari-ppo.yaml

config = (
    PPOConfig()
    .environment(env=env_name)
    .framework(framework="torch")
    .rollouts(
        num_rollout_workers=4, rollout_fragment_length=128,
        # num_envs_per_worker=5,
        # batch_mode="truncate_episodes",
    )
    .training(
        train_batch_size=512,
        lr=2e-5,
        gamma=0.99,
        lambda_=0.9,
        use_gae=True,
        clip_param=0.4,
        grad_clip=None,
        entropy_coeff=0.1,
        vf_loss_coeff=0.25,
        sgd_minibatch_size=64,
        num_sgd_iter=10,
        # lambda_=0.95,
        # kl_coeff=0.5,
        # clip_param=0.1,
        # vf_clip_param=10.0,
        # entropy_coeff=0.01,
        # train_batch_size=5000,
        # sgd_minibatch_size=500,
        # num_sgd_iter=10,
    )
    # .evaluation(
    #     evaluation_duration=3600,
    #     evaluation_duration_unit="timesteps",
    #     evaluation_num_workers=1,
    # )
    .debugging(log_level="WARN", seed=SEED)
    .resources(num_gpus=int(os.environ.get("RLLIB_NUM_GPUS", "0")))
)

In [6]:
from time import time
from ray import tune

current_time = int(time())
 
tune.run(
    "PPO",
    name="PPO",
    stop={"timesteps_total": 2e4},
    checkpoint_freq=10,
    local_dir=os.path.join("ray_results", env_name, str(current_time)),
    config=config.to_dict(),
)

c:\Users\admin\anaconda3\envs\marl\lib\site-packages\ray\tune\experiment\experiment.py:170: UserWarning: The `local_dir` argument of `Experiment is deprecated. Use `storage_path` or set the `TUNE_RESULT_DIR` environment variable instead.
  warnings.warn(


(PPO pid=19016) 2023-05-29 12:15:08,021	WARNING algorithm_config.py:635 -- Cannot create PPOConfig from given `config_dict`! Property __stdout_file__ not supported.
(PPO pid=19016) 2023-05-29 12:15:08,491	INFO algorithm.py:527 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.


(RolloutWorker pid=23028) Step #0.00 (0ms ?*RT. ?UPS, TraCI: 66ms, vehicles TOT 0 ACT 0 BUF 0)                     


(RolloutWorker pid=7864) 2023-05-29 12:15:15,244	WARNING env.py:285 -- Your MultiAgentEnv <ParallelPettingZooEnv instance> does not have some or all of the needed base-class attributes! Make sure you call `super().__init__()` from within your MutiAgentEnv's constructor. This will raise an error in the future.
(RolloutWorker pid=7864) Warning: Vehicle '0' performs emergency braking on lane ':D3_45_0' with decel=9.00, wished=4.50, severity=1.00, time=142.00.


Step #100.00 (0ms ?*RT. ?UPS, TraCI: 110ms, vehicles TOT 21 ACT 21 BUF 0)                 OT 1 ACT 1 BUF 0)                     
(RolloutWorker pid=7864) Step #0.00 (0ms ?*RT. ?UPS, TraCI: 191ms, vehicles TOT 0 ACT 0 BUF 0)                     [repeated 11x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#log-deduplication for more options.)


...

Step #1900.00 (1ms ~= 1000.00*RT, ~230000.00UPS, TraCI: 170ms, vehicles TOT 824 ACT 230 BUT 1 ACT 1 BUF 0)                      


(RolloutWorker pid=3380) Warning: Vehicle '1116' performs emergency braking on lane ':C0_39_0' with decel=9.00, wished=4.50, severity=1.00, time=2414.00.


Step #2400.00 (0ms ?*RT. ?UPS, TraCI: 130ms, vehicles TOT 1136 ACT 118 BUF 0)             CI: 134ms, vehicles TOT 551 ACT 133 BU
Step #3600.00 (0ms ?*RT. ?UPS, TraCI: 149ms, vehicles TOT 1473 ACT 36 BUF 0)              OT 1 ACT 1 BUF 0)                      
Step #3600.00 (0ms ?*RT. ?UPS, TraCI: 153ms, vehicles TOT 1473 ACT 36 BUF 0)              T 1 ACT 1 BUF 0)                      
Step #3600.00 (1ms ~= 1000.00*RT, ~35000.00UPS, TraCI: 141ms, vehicles TOT 1473 ACT 35 BUFCI: 126ms, vehicles TOT 1175 ACT 101 B
Step #3600.00 (1ms ~= 1000.00*RT, ~39000.00UPS, TraCI: 138ms, vehicles TOT 1473 ACT 39 BUFI: 163ms, vehicles TOT 907 ACT 216 BUF


2023-05-29 12:40:05,537	INFO tune.py:945 -- Total run time: 1502.44 seconds (1501.42 seconds for the tuning loop).


Step #405.00 (1ms ~= 1000.00*RT, ~44000.00UPS, TraCI: 5887ms, vehicles TOT 85 ACT 44 BUF 0OT 1 ACT 1 BUF 0)                      
Step #405.00 (0ms ?*RT. ?UPS, TraCI: 5768ms, vehicles TOT 85 ACT 40 BUF 0)                T 1 ACT 1 BUF 0)                      
Step #405.00 (0ms ?*RT. ?UPS, TraCI: 5748ms, vehicles TOT 85 ACT 38 BUF 0)                T 1 ACT 1 BUF 0)                      
Step #405.00 (0ms ?*RT. ?UPS, TraCI: 5897ms, vehicles TOT 85 ACT 45 BUF 0)                T 1 ACT 1 BUF 0)                      


In [7]:
ray.shutdown()

## Watching the Trained Agent

## RLlib Docs

In [ ]:
algo = config.build()

In [ ]:
# Changes made in ParallelPettingZooEnv's reset() method: see line 202-206 in
# C:\Users\admin\anaconda3\envs\marl\lib\site-packages\ray\rllib\env\wrappers\pettingzoo_env.py

algo.evaluate()

In [ ]:
from ray.tune.logger import pretty_print

result = algo.train()
print(pretty_print(result))

In [ ]:
algo.evaluate()